## Resources
[NLTK][https://stackabuse.com/text-classification-with-python-and-scikit-learn/]

In [1]:
import nltk
import pickle
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics import accuracy_score, precision_score, classification_report
from sklearn.ensemble import RandomForestClassifier

# nltk.download('stopwords')
# nltk.download('wordnet')

In [3]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

plt.rcParams['figure.figsize'] = (9, 6)
sns.set(context='notebook', style='whitegrid', font_scale=1.2)

In [24]:
from functions import load_data_set
myers_briggs = load_data_set()
mb_df = pd.DataFrame(myers_briggs)

In [26]:
X, y = mb_df[1], mb_df[0]

## Might want to remove URLs

In [28]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [29]:
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [30]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [32]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train) 

RandomForestClassifier(n_estimators=1000, random_state=0)

In [33]:
y_pred = classifier.predict(X_test)

In [34]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[  0   1   0   0   0   0   0   0   6  21   4   3   0   0   0   0]
 [  0  65   0   2   0   0   0   0  13  36  10   9   0   0   0   1]
 [  0   1   7   2   0   0   0   0   6  13   7   4   0   0   0   0]
 [  0  10   0  61   0   0   0   0  14  21   9  29   0   0   1   0]
 [  0   0   0   1   0   0   0   0   3   5   1   3   0   0   0   0]
 [  0   1   0   0   0   0   0   0   3   4   0   1   0   0   0   0]
 [  0   1   1   0   0   0   0   0   2   3   0   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0   3   5   0   3   0   0   0   0]
 [  0   4   0   2   0   0   0   0 222  63   4  10   0   0   0   0]
 [  1   3   0   1   0   0   0   0  16 306   3  12   0   0   0   0]
 [  0   2   0   3   0   0   0   0  24  41 136  32   0   0   1   0]
 [  0   2   0   3   0   0   0   0  11  42   6 213   0   0   0   0]
 [  0   0   0   0   0   0   0   0   4  17   4   5   3   0   0   0]
 [  0   2   0   1   0   0   0   0   7  35   1   7   0   3   0   0]
 [  0   2   0   2   0   0   0   0   2   7   3   6   0   0   4 

In [35]:
with open('text_classifier', 'wb') as picklefile:
    pickle.dump(classifier,picklefile)

In [36]:
with open('text_classifier', 'rb') as training_model:
    model = pickle.load(training_model)

In [37]:
y_pred2 = model.predict(X_test)

print(confusion_matrix(y_test, y_pred2))
print(classification_report(y_test, y_pred2))
print(accuracy_score(y_test, y_pred2)) 

[[  0   1   0   0   0   0   0   0   6  21   4   3   0   0   0   0]
 [  0  65   0   2   0   0   0   0  13  36  10   9   0   0   0   1]
 [  0   1   7   2   0   0   0   0   6  13   7   4   0   0   0   0]
 [  0  10   0  61   0   0   0   0  14  21   9  29   0   0   1   0]
 [  0   0   0   1   0   0   0   0   3   5   1   3   0   0   0   0]
 [  0   1   0   0   0   0   0   0   3   4   0   1   0   0   0   0]
 [  0   1   1   0   0   0   0   0   2   3   0   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0   3   5   0   3   0   0   0   0]
 [  0   4   0   2   0   0   0   0 222  63   4  10   0   0   0   0]
 [  1   3   0   1   0   0   0   0  16 306   3  12   0   0   0   0]
 [  0   2   0   3   0   0   0   0  24  41 136  32   0   0   1   0]
 [  0   2   0   3   0   0   0   0  11  42   6 213   0   0   0   0]
 [  0   0   0   0   0   0   0   0   4  17   4   5   3   0   0   0]
 [  0   2   0   1   0   0   0   0   7  35   1   7   0   3   0   0]
 [  0   2   0   2   0   0   0   0   2   7   3   6   0   0   4 

In [ ]:
# Create the  pipeline to clean, tokenize, vectorize, and classify 
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])

# Create model and measure accuracy
pipe.fit([x[0] for x in train], [x[1] for x in train]) 
pred_data = pipe.predict([x[0] for x in test]) 
for (sample, pred) in zip(test, pred_data):
    print(sample, pred)
print("Accuracy:", accuracy_score([x[1] for x in test], pred_data))

In [ ]:
print("Precision:", precision_score([x[1] for x in test], pred_data, average='micro'))

In [ ]:
print("Precision:", precision_score([x[1] for x in test], pred_data, average=None))

In [ ]:
cr = classification_report([x[1] for x in test], pred_data)

In [ ]:
cr.split('\n')